In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 필요 라이브러리 import

In [15]:
import pandas as pd
import numpy as np

# 데이터 불러오기
1) 제공받은 pdde, cop, lpay 파일 불러오기</br>
2) 날씨 데이터 불러오기

In [37]:
demo = pd.read_csv('/content/drive/MyDrive/롯데멤버스_빅데이터경진대회/LPOINT_BIG_COMP_01_DEMO.csv')
pdde = pd.read_csv('/content/drive/MyDrive/롯데멤버스_빅데이터경진대회/LPOINT_BIG_COMP_02_PDDE.csv')
cop_u = pd.read_csv('/content/drive/MyDrive/롯데멤버스_빅데이터경진대회/LPOINT_BIG_COMP_03_COP_U.csv')
pd_clac = pd.read_csv('/content/drive/MyDrive/롯데멤버스_빅데이터경진대회/LPOINT_BIG_COMP_04_PD_CLAC.csv')
br = pd.read_csv('/content/drive/MyDrive/롯데멤버스_빅데이터경진대회/LPOINT_BIG_COMP_05_BR.csv')
lpay = pd.read_csv('/content/drive/MyDrive/롯데멤버스_빅데이터경진대회/LPOINT_BIG_COMP_06_LPAY.csv')

# 날씨 데이터
# 날씨 데이터 불러오기(전국 날씨를 기준으로)
강수량 = pd.read_csv('/content/drive/MyDrive/롯데멤버스_빅데이터경진대회/2021_전국_강수량 (1).csv',encoding = 'cp949')
기온 = pd.read_csv('/content/drive/MyDrive/롯데멤버스_빅데이터경진대회/2021_전국_기온.csv',encoding = 'cp949')
습도 = pd.read_csv('/content/drive/MyDrive/롯데멤버스_빅데이터경진대회/2021_전국_습도.csv',encoding = 'cp949')
일조량 = pd.read_csv('/content/drive/MyDrive/롯데멤버스_빅데이터경진대회/2021_전국_일조량.csv',encoding = 'cp949')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# 데이터프레임 병합

### 상품구매 데이터 병합
- PDDE + DEMO + PD_CLAC + br

In [38]:
#상품 구매 정보(pdde) + 고객정보(demo)
pdde_demo = pd.merge(demo, pdde, left_on = 'cust', right_on='cust', how='right') # cust(고객번호)를 기준으로 right조인

#상품 구매정보(pdde) + 고객 정보(demo) + 상품 분류 정보(pd_clac)
pdde_demo_pd_clac = pd.merge(pdde_demo, pd_clac, left_on = 'pd_c', right_on = 'pd_c', how = 'left')

#상품 구매정보(pdde) + 고객 정보(demo) + 상품 분류 정보(pd_clac) + 점포 정보(br)

pdde_merge = pd.merge(pdde_demo_pd_clac, br , on = ['cop_c', 'br_c', 'zon_hlv'], how='left')

In [39]:
pdde_merge.head(3)

,cust,ma_fem_dv,ages,zon_hlv,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,pd_nm,clac_hlv_nm,clac_mcls_nm,zon_mcls
0,M430112881,여성,50대,Z17,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1,남성티셔츠,남성의류,남성의류상의,NaN
1,M646853852,여성,40대,Z13,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1,기타주방일회용품,주방잡화,주방일회용품,Z13001
2,M430112881,여성,50대,Z17,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1,남성티셔츠,남성의류,남성의류상의,NaN


### 제휴사 데이터 병합
COP_U + DEMO + BR

In [40]:
tmp = pd.merge(cop_u,demo, left_on = 'cust', right_on='cust', how='left')
cop_u_merge = pd.merge(tmp, br, on = ['cop_c','br_c', 'zon_hlv'], how='left')

In [41]:
cop_u_merge.head(3)

,cust,rct_no,cop_c,br_c,chnl_dv,de_dt,vst_dt,de_hr,buy_am,ma_fem_dv,ages,zon_hlv,zon_mcls
0,M839993508,21102612B015763935,B01,B010012,1,20211026,20211026,12,60000,남성,70대,Z17,Z17024
1,M839993508,21110610B014219744,B01,B010012,1,20211106,20211106,10,17100,남성,70대,Z17,Z17024
2,M839993508,21021112B013419710,B01,B010012,1,20210211,20210211,12,136500,남성,70대,Z17,Z17024


### 엘페이 데이터 병합
LPAY + DEMO

In [42]:
lpay_merge = pd.merge(lpay, demo, how = 'left', on='cust')

# 날씨 데이터 병합

In [43]:
# 조인하기 전 데이터 전처리
print(len(강수량))
print(len(기온))
print(len(습도)) # 습도는 뒤에 뭐가 더 붙었다. 조인으로 해줘야겠다 concat쓰면 잘못될 수 있음.
print(len(일조량))

365
365
371
365


In [45]:
# weather로 4개 csv 날씨 데이터 merge
weather = pd.merge(강수량, 기온, how = 'left',on = '날짜')
습도.drop(['Unnamed: 3'	,'Unnamed: 4'], axis=1, inplace=True)
습도.drop([365, 366,367,368,369,370], inplace=True)
습도['일시'] = 습도['일시'].apply(lambda x: x.strip()) # 일시 앞에 공백이 있어서 안지워주면 조인이 안된다.
weather = pd.merge(weather, 습도, how='left', left_on='날짜', right_on='일시')
일조량['일시'] = 일조량['일시'].apply(lambda x: x.strip())
weather = pd.merge(weather, 일조량, how='left', on='일시')
weather.drop('일시',axis=1, inplace=True)
weather['날짜'] = pd.to_datetime(weather['날짜'])



# pdde에 조인
pdde_merge['de_dt'] = pdde_merge['de_dt'].astype(str)
pdde_merge['de_dt'] = pd.to_datetime(pdde_merge['de_dt'])

pdde_merge = pd.merge(pdde_merge, weather, how='left', left_on='de_dt', right_on='날짜')
pdde_merge.drop(['날짜'], axis=1, inplace = True)



# cop에 조인
cop_u_merge['de_dt'] = cop_u_merge['de_dt'].astype(str)
cop_u_merge['de_dt'] = pd.to_datetime(cop_u_merge['de_dt'])

cop_u_merge = pd.merge(cop_u_merge, weather, how='left', left_on='de_dt', right_on='날짜')
cop_u_merge.drop(['날짜'], axis=1, inplace = True)



# lpay에 조인
lpay_merge['de_dt'] = lpay_merge['de_dt'].astype(str)
lpay_merge['de_dt'] = pd.to_datetime(lpay_merge['de_dt'])

lpay_merge = pd.merge(lpay_merge, weather, how='left', left_on='de_dt', right_on='날짜')
lpay_merge.drop(['날짜'], axis=1, inplace = True)


In [58]:
# CSV 파일로 내보내기
pdde_merge.to_csv('pdde_final.csv')
cop_u_merge.to_csv('cop_final.csv')
lpay_merge.to_csv('lpay_final.csv')

# 데이터 전처리 함수 만들기
- 모든 데이터프레임에 적용가능하도록 하나의 전처리 함수 작성</br>
1) 날짜 파생변수(연, 월, 일) 생성</br>
2) 일교차, 불쾌지수 생성(날씨 파생변수)</br>
3) 기념일 정보(공휴일, 주말 포함)


In [50]:
def preprocess(df):
    #성별 남자 : 0, 여성 1
    #나이 : 20대 0 30대 1 40대 2 50대 3 60대 4 70대 5
    # df.replace({'ma_fem_dv': {"남성": 0, "여성": 1},
    #                   'ages': {"20대": 0,"30대": 1, "40대": 2, "50대": 3, "60대": 4, "70대": 5 }},inplace=True)

    #날짜 split
    df['de_dt'] = df['de_dt'].astype(str)
    df['de_dt'] = pd.to_datetime(df['de_dt'])
    df['year'] = df['de_dt'].apply(lambda x: x.year)
    df['month'] = df['de_dt'].apply(lambda x: x.month)
    df['요일'] = df['de_dt'].apply(lambda x: x.weekday())
    
    #일교차 생성
    df['일교차'] = df["최고기온(℃)"] - df["최저기온(℃)"]

    # 불쾌지수 생성
    df['불쾌지수'] = 9/5*df['평균기온(℃)']-0.55*(1-df['평균습도(%rh)']/100)*(9/5*df['평균기온(℃)']-26) + 32



    # 2020년 데이터 제외
    df = df[df['year']!=2020]
    df= df.reset_index(drop=True)



    #공휴일 생성 요일은 월:0, 화:1
    tmp = []
    # 주말
    for i in range(len(df)):
        if df['요일'][i]==5 or df['요일'][i]==6: # 주말이면
            tmp.append(df['de_dt'][i])
    # 21년도 기준 공휴일 & 기념일 & 대체휴일
    공휴일 = ['2021-01-01','2021-02-11','2021-02-12', '2021-02-13','2021-02-14','2021-03-01','2021-03-14','2021-05-05','2021-05-08','2021-06-06',
             '2021-08-15','2021-08-16','2021-09-20','2021-09-21','2021-09-22','2021-10-03','2021-10-04','2021-10-09','2021-10-11','2021-12-24','2021-12-25']

    tmp.extend(공휴일)

    tmp = list(set(tmp)) #중복제거

    holiday_anniversary = []
    for i in range(len(df)):
        if df['de_dt'][i] in tmp:
            holiday_anniversary.append(1)
        else:
            holiday_anniversary.append(0)

    df['holiday_anniversary'] = holiday_anniversary
    return df